## Fazendo Web Scraping do Top 250 filmes do IMDb

In [ ]:
import time, random
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # Ajuda a gerenciar o chromedriver

# Configura o driver do Selenium para controlar o Chrome
# O ChromeDriverManager baixa e gerencia o driver automaticamente
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# URL do nosso alvo
url = 'https://www.imdb.com/chart/top/'

# Selenium abre a página no navegador
driver.get(url)

# Espera inicial para a página carregar
time.sleep(3)

# Rola a página para baixo para carregar todos os filmes dinamicamente
while True:
    # Conta quantos filmes estão carregados na página
    filmes_carregados = len(driver.find_elements(By.CSS_SELECTOR, 'ul.ipc-metadata-list li'))

    if filmes_carregados >= 250:
        print('Todos os 250 filmes foram carregados.')
        break

    # Executa um script para rolar até o final da página
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # Espera um pouco para o novo conteúdo carregar
    time.sleep(2)

# Pega o HTML completo da página depois que tudo foi carregado
html_completo = driver.page_source

# Fecha o navegador controlado pelo Selenium
driver.quit()


Todos os 250 filmes foram carregados.


In [26]:
# Definindo 'headers' para simular um navegador
# Alguns sites bloqueiam requisições que não parecem vir de um navegador real.
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36'
}

# Criando uma sessão persistente
# Reaproveita cookies/conexões e aplica headers em todas as requisições
session = requests.Session()
session.headers.update(headers)

# Função auxiliar para extrair o texto de um elemento do BeautifulSoup
def get_text(el):
    return el.get_text(strip=True) if el else None

# Função para fazer a requisição HTTP com várias tentativas (retries)
def fetch(url, retries=3, backoff=2):
    for i in range(retries):
        try:
            r = session.get(url, timeout=20) # Timeout evita travar se o servidor não responder
            if r.status_code == 200: # Se resposta OK, retorna imediatamente
                return r
            print(f'Tentativa {i+1} falhou para {url} com status {r.status_code}')
        except requests.exceptions.RequestException as e:
            print(f'Tentativa {i+1} falhou para {url} com erro: {e}')
        time.sleep(backoff ** i) # Espera antes da próxima tentativa
    raise RuntimeError(f'Falha ao baixar: {url}') # Se todas as tentativas falharem, lança erro

# Analisando (parsing) o conteúdo HTML COMPLETO com o BeautifulSoup
# A única mudança aqui é usar 'html_completo' vindo do Selenium
soup = BeautifulSoup(html_completo, 'html.parser')

# Encontrando a lista de filmes
filmes_data = []
# A investigação mostrou que cada filme está em uma tag <li> dentro de uma <ul> com a classe 'ipc-metadata-list'
lista_filmes = soup.find('ul', class_='ipc-metadata-list')

print('\nIniciando a extração dos dados de cada filme...')
for i, filme_li in enumerate(lista_filmes.find_all('li')):
    try:
        # Dados básicos do filme na primeira página
        # Extraindo Título, Ano e Nota
        titulo_tag = filme_li.find('h3', class_='ipc-title__text')
        if not titulo_tag:
            continue
        titulo = titulo_tag.text.split('. ', 1)[1]
        print(f'Extraindo dados do filme {i+1}/250: {titulo}')

        metadata = filme_li.find_all('span', class_='cli-title-metadata-item')
        ano = metadata[0].text if metadata else None

        nota_tag = filme_li.find('span', class_='ipc-rating-star')
        nota = get_text(nota_tag).split('(')[0].strip() if nota_tag else None

        # Extraindo o LINK para a página do filme específico
        link_relativo = filme_li.find('a')['href']
        link_completo = 'https://www.imdb.com' + link_relativo

        # Acessando a página individual do filme para mais detalhes
        time.sleep(random.uniform(1.0, 2.0))  # Pausa para respeitar o site
        response_filme = fetch(link_completo)
        soup_filme = BeautifulSoup(response_filme.content, 'html.parser')

        # Sinopse (fallbacks pois o data-testid muda às vezes)
        sinopse = None
        for sel in [
                '[data-testid="plot-xl"]',
                '[data-testid="plot-l"]',
                'span[data-testid="plot-xl"]',
                'span[data-testid="plot"]',
                'div[data-testid="storyline-plot-summary"] > div > div'
            ]:
            el = soup_filme.select_one(sel)
            if el:
                sinopse = get_text(el)
                break

        # Elenco principal
        elenco_tags = soup_filme.select("a[data-testid='title-cast-item__actor']")
        elenco = [get_text(a) for a in elenco_tags][:3]

        # Diretor
        diretor = None
        creditos_tags = soup_filme.find_all('ul', {'role': 'presentation', 'class': 'ipc-metadata-list'})
        if creditos_tags:
            a0 = creditos_tags[0].find('a')
            diretor = get_text(a0)

        filmes_data.append({
            'Título': titulo,
            'Ano': ano,
            'Nota': nota,
            'Sinopse': sinopse,
            'Diretor': diretor,
            'Elenco Principal': ', '.join(elenco) # Juntando o elenco em uma string
        })

    except Exception as e:
        print(f"Falha ao extrair dados de '{titulo if 'titulo' in locals() else 'Desconhecido'}': {e}")

print('\nColeta finalizada!')
print(f'Total de filmes extraídos: {len(filmes_data)}')

for filme in filmes_data[:5]:
    print(filme)


Iniciando a extração dos dados de cada filme...
Extraindo dados do filme 1/250: Um Sonho de Liberdade
Extraindo dados do filme 2/250: O Poderoso Chefão
Extraindo dados do filme 3/250: Batman: O Cavaleiro das Trevas
Extraindo dados do filme 4/250: O Poderoso Chefão: Parte II
Extraindo dados do filme 5/250: 12 Homens e uma Sentença
Extraindo dados do filme 6/250: O Senhor dos Anéis: O Retorno do Rei
Extraindo dados do filme 7/250: A Lista de Schindler
Extraindo dados do filme 8/250: O Senhor dos Anéis: A Sociedade do Anel
Extraindo dados do filme 9/250: Pulp Fiction: Tempo de Violência
Extraindo dados do filme 10/250: Três Homens em Conflito
Extraindo dados do filme 11/250: Forrest Gump: O Contador de Histórias
Extraindo dados do filme 12/250: O Senhor dos Anéis: As Duas Torres
Extraindo dados do filme 13/250: Clube da Luta
Extraindo dados do filme 14/250: A Origem
Extraindo dados do filme 15/250: Star Wars: Episódio V - O Império Contra-Ataca
Extraindo dados do filme 16/250: Matrix
Ext

In [27]:
df_filmes = pd.DataFrame(filmes_data)

caminho_arquivo = '../data/raw/imdb_top_250.csv'
df_filmes.to_csv(caminho_arquivo, index=False,
                                  sep = ';',
                                  decimal = ',',
                                  encoding = 'latin1'
)

print(f"\nDados salvos com sucesso em: {caminho_arquivo}")

df_filmes.head(10)


Dados salvos com sucesso em: ../data/raw/imdb_top_250.csv


,Título,Ano,Nota,Sinopse,Diretor,Elenco Principal
0,Um Sonho de Liberdade,1994,"9,3",Dois homens presos se reúnem ao longo de vário...,Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton"
1,O Poderoso Chefão,1972,"9,2",O patriarca idoso de uma dinastia do crime org...,Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan"
2,Batman: O Cavaleiro das Trevas,2008,"9,1",Agora com a ajuda do tenente Jim Gordon e do p...,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart"
3,O Poderoso Chefão: Parte II,1974,"9,0",O início da vida e da carreira de Vito Corleon...,Francis Ford Coppola,"Al Pacino, Robert De Niro, Robert Duvall"
4,12 Homens e uma Sentença,1957,"9,0",O julgamento de um assassinato em Nova Iorque ...,Sidney Lumet,"Henry Fonda, Lee J. Cobb, Martin Balsam"
5,O Senhor dos Anéis: O Retorno do Rei,2003,"9,0",Gandalf e Aragorn lideram o Mundo dos Homens c...,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen"
6,A Lista de Schindler,1993,"9,0",Na Polônia ocupada pelos alemães durante a Seg...,Steven Spielberg,"Liam Neeson, Ralph Fiennes, Ben Kingsley"
7,O Senhor dos Anéis: A Sociedade do Anel,2001,"8,9",Um manso hobbit do Condado e oito companheiros...,Peter Jackson,"Elijah Wood, Ian McKellen, Orlando Bloom"
8,Pulp Fiction: Tempo de Violência,1994,"8,8","As vidas de dois assassinos da máfia, um boxea...",Quentin Tarantino,"John Travolta, Uma Thurman, Samuel L. Jackson"
9,Três Homens em Conflito,1966,"8,8",Um impostor se junta com dois homens para enco...,Sergio Leone,"Clint Eastwood, Eli Wallach, Lee Van Cleef"
